In [2]:
%pip install selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install undetected-chromedriver

In [1]:
from webdriver_manager.chrome import ChromeDriverManager #한달마다 크롬 업데이트 되니까 그때마다 한 번씩 더 실행시켜기기

ChromeDriverManager().install()

'C:\\Users\\gfddf\\.wdm\\drivers\\chromedriver\\win64\\135.0.7049.84\\chromedriver-win32/chromedriver.exe'

In [2]:
from selenium import webdriver
import undetected_chromedriver as uc

options = uc.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")  # 탐지 우회
driver = uc.Chrome(version_main=134, options=options)

url = 'https://www.instacart.com/store/?categoryFilter=homeTabForYou'

driver.get(url)

In [15]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

aisles = [
    "eggs",
    "fresh vegetables",
    "spices seasonings",
    "oils vinegars",
    "baking ingredients",
    "doughs gelatins bake mixes",
    "spreads",
    "packaged vegetables fruits",
    "yogurt",
    "soy lactosefree",
    "poultry counter",
    "bread",
    "breakfast bakery",
    "cold flu allergy",
    "energy granola bars",
    "breakfast bars pastries",
    "chips pretzels",
    "trail mix snack mix",
    "crackers",
    "refrigerated",
    "energy sports drinks",
    "fresh fruits",
    "salad dressing toppings",
    "prepared soups salads",
    "milk",
    "paper goods",
    "water seltzer sparkling water",
    "kosher foods",
    "packaged poultry",
    "packaged cheese",
    "instant foods",
    "packaged produce",
    "cookies cakes",
    "candy chocolate",
    "body lotions soap",
    "dry pasta",
    "laundry",
    "air fresheners candles",
    "frozen produce",
    "buns rolls",
    "canned fruit applesauce",
    "juice nectars",
    "granola"
]

In [16]:
def XPATH_input(i, j):
    a = f'//*[@id="store-wrapper"]/div/div/ul/li[{i}]/div[1]/div/div/div/div/div[2]/ul/div[{j}]/li/div/div/h3/div/a/div[2]/div/div[1]/div[1]/div[1]/span[2]/span[2]'
    b = f'//*[@id="store-wrapper"]/div/div/ul/li[{i}]/div[1]/div/div/div/div/div[2]/ul/div[{j}]/li/div/div/h3/div/a/div[2]/div/div[1]/div[1]/div[1]/span[2]/span[3]'
    return a, b

def product_price_li():
    product_price = []

    i = 1
    stop_flag = True
    scroll_count = 0

    while stop_flag:
        for j in range(1,7):
            a, b = XPATH_input(i,j)
            try:
                # prod_a = driver.find_element(By.XPATH, a).text
                # prod_b = driver.find_element(By.XPATH, b).text
                prod_a = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.XPATH, a))
                ).text
                prod_b = WebDriverWait(driver, 2).until(
                    EC.presence_of_element_located((By.XPATH, b))
                ).text
                prod_merge = float(prod_a+'.'+prod_b)
                product_price.append(prod_merge)
            except:
                continue
            if len(product_price) > 14:
                stop_flag= False
                break
        i+=1

        # 더 이상 아이템이 안 보이면 스크롤 실행 (최대 5번)
        if stop_flag and scroll_count < 3:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            scroll_count += 1
        else:
            stop_flag = False
            
    if product_price: 
        product_mean = sum(product_price)/len(product_price)
    else:
        product_mean = None
    return product_mean

In [17]:
products_mean = {}

for aisle in aisles:
    driver.find_element(By.ID, 'search-bar-input').send_keys(aisle)
    driver.find_element(By.ID, 'search-bar-input').send_keys(Keys.ENTER)
    time.sleep(2)
    
    try:
        clear_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "e-7oyql3"))
        )
        clear_button.click()
    except:
        print("삭제 버튼을 찾을 수 없음")

    data = product_price_li()
    if data:
        products_mean[aisle] = data
        print(aisle, data)

eggs 7.618571428571428
oils vinegars 5.67
baking ingredients 0.92
prepared soups salads 7.864999999999999
